In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import autoregressive

In [2]:
# chess

with open("chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)

# from collections import Counter
# vocabulary = Counter(text)

# from nltk.lm import Vocabulary
# vocabulary = Vocabulary(text)

len(vocabulary)

92

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [6]:
length = 17
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])

556949
556932
['\ufeffThe Project Gute', 'The Project Guten', 'he Project Gutenb', 'e Project Gutenbe', ' Project Gutenber']


# fit

In [32]:
importlib.reload(autoregressive)    
net = autoregressive.LSTM(vocabulary)
# net = autoregressive.TransformerEncoder(vocabulary)
net.to(device)

encoded = net.text2tensor(lines)
print("encoded", encoded.shape)

performance = net.fit(encoded, save_path = "character_model.pt")
net.save_architecture("character_model.arch")

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

encoded torch.Size([556932, 17])


  0%|          | 0/5 [00:00<?, ?it/s]

Training started
Epochs: 5
Learning rate: 0.0001
Weight decay: 0
Epoch | Train                 | Minutes
      | Loss     | Error Rate |
---------------------------------------


  0%|          | 0/5570 [00:00<?, ?it/s]

    1 |   1.9529 |     49.411 |     0.9


...

    3 |   1.2030 |     34.457 |     2.8


  0%|          | 0/5570 [00:00<?, ?it/s]

    4 |   1.1348 |     32.858 |     3.5


  0%|          | 0/5570 [00:00<?, ?it/s]

    5 |   1.0912 |     31.802 |     4.2


In [40]:
# includes all the information about the epoch and the model, useful for reproducibility

performance

,epoch,train_loss,train_error_rate,training_minutes,learning_rate,weight_decay,model,embedding_dimension,hidden_units,layers,dropout,parameters
0,1,1.952884,49.411104,0.884901,0.0001,0,Autoregressive LSTM,32,128,2,0.0,230335
1,2,1.338458,37.345493,1.904391,0.0001,0,Autoregressive LSTM,32,128,2,0.0,230335
2,3,1.203032,34.457113,2.793132,0.0001,0,Autoregressive LSTM,32,128,2,0.0,230335
3,4,1.134803,32.858163,3.480790,0.0001,0,Autoregressive LSTM,32,128,2,0.0,230335
4,5,1.091227,31.801829,4.163907,0.0001,0,Autoregressive LSTM,32,128,2,0.0,230335


In [41]:
# the input for testing

net.tensor2text(encoded[:5])

['\ufeffThe Project Gute',
 'The Project Guten',
 'he Project Gutenb',
 'e Project Gutenbe',
 ' Project Gutenber']

# predict

In [42]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("character_model.arch")
net.load_state_dict(torch.load("character_model.pt"))
net.to(device)
idx, log_probabilities = net.predict(encoded[:5], main_progress_bar = False, progress_bar = 0)

net.tensor2text(idx)

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335



['\ufeffThe Project Gutenberg.\n\n        ----',
 'The Project Gutenberg.\n\n        -----',
 'he Project Gutenberg.\n\n        ------',
 'e Project Gutenberg.\n\n        -------',
 ' Project Gutenberg.\n\n        --------']

In [43]:
log_probabilities

tensor([[ -7.2597,  -7.6449,  -8.5382,  -8.7951,  -8.8346],
        [ -7.1597,  -7.6021,  -8.4619,  -8.7094,  -8.7485],
        [ -6.6665,  -7.1435,  -8.6905,  -9.1799,  -9.3141],
        [ -6.5764,  -7.1233,  -8.7336,  -9.2426,  -9.3799],
        [ -6.4960,  -7.2014,  -9.5186,  -9.6730, -10.0534]], device='cuda:0')

# greedy_search

In [44]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("character_model.arch")
net.load_state_dict(torch.load("character_model.pt"))
net.to(device)
indexes, log_probabilities = net.greedy_search(encoded[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([-10.3600, -10.2795, -10.2705, -11.9845, -12.8392], device='cuda:0')


['\ufeffThe Project Gutenberg-tm and the paw',
 'The Project Gutenberg-tm and the pawn',
 'he Project Gutenberg-tm and the pawn ',
 'e Project Gutenberg-tm and the pawn a',
 ' Project Gutenberg-tm and the pawn at']

# sample

In [45]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("character_model.arch")
net.load_state_dict(torch.load("character_model.pt"))
net.to(device)
indexes, log_probabilities = net.sample(encoded[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([-19.8109, -22.6527, -31.2681, -17.7597, -24.2398], device='cuda:0')


['\ufeffThe Project Gutenberg mowen by Kt-Kt',
 'The Project Gutenberg B, and Kt-B5ch ',
 "he Project Gutenberg. Black's porock,",
 'e Project Gutenberg\nR5.\n\n         11.',
 ' Project Gutenberg. The bettime his B']

# beam_search

In [46]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("character_model.arch")
net.load_state_dict(torch.load("character_model.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(encoded[:5], progress_bar = False)

print(log_probabilities)
pprint([net.tensor2text(t) for t in indexes])

Model: Autoregressive LSTM
Tokens in the vocabulary: 92
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([[ -7.2597,  -7.6449,  -8.5382,  -8.7951,  -8.8346],
        [ -7.1597,  -7.6021,  -8.4619,  -8.7094,  -8.7485],
        [ -6.6665,  -7.1435,  -8.6905,  -9.1799,  -9.3141],
        [ -6.5764,  -7.1233,  -8.7336,  -9.2426,  -9.3799],
        [ -6.4960,  -7.2014,  -9.5186,  -9.6730, -10.0534]], device='cuda:0')
[['\ufeffThe Project Gutenberg.\n\n        ----',
  '\ufeffThe Project Gutenberg.\n\n            ',
  '\ufeffThe Project Gutenberg.\n\n          1.',
  '\ufeffThe Project Gutenberg.\n\n          2.',
  '\ufeffThe Project Gutenberg.\n\n          3.'],
 ['The Project Gutenberg.\n\n        -----',
  'The Project Gutenberg.\n\n             ',
  'The Project Gutenberg.\n\n          1. ',
  'The Project Gutenberg.\n\n          2. ',
  'The Project Gutenberg.\n\n          3. '],
 ['he Project Gutenberg.\n\n        ------',
  'he Pr